In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd


In [2]:
data = pd.read_csv('housing-data.csv')


In [18]:
X = data[['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','PTRATIO','B','LSTAT']]

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     486 non-null    float64
 1   ZN       486 non-null    float64
 2   INDUS    486 non-null    float64
 3   CHAS     486 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      486 non-null    float64
 7   DIS      506 non-null    float64
 8   PTRATIO  506 non-null    float64
 9   B        506 non-null    float64
 10  LSTAT    486 non-null    float64
dtypes: float64(11)
memory usage: 43.6 KB


In [20]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,18.7,396.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,21.0,391.99,NaN
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,21.0,393.45,6.48


In [21]:
y = data[['MEDV']]

In [6]:
y

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos (recomendable para redes neuronales)
#scaler = StandardScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Añadir dimensión
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [42]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)  # Capa de entrada a capa oculta
        self.fc2 = nn.Linear(64, 32)  # Segunda capa oculta
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activación ReLU
        x = torch.relu(self.fc2(x))  # Activación ReLU
        x = self.fc3(x)              # No se aplica activación en la capa de salida
        return x

# Crear una instancia del modelo
model = NeuralNet(12,1)


In [35]:
criterion = nn.MSELoss()

# Definir el optimizador (Adam en este caso)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Definir hiperparámetros
epochs = 100
batch_size = 32

# Dividir los datos en batches
train_loader = torch.utils.data.DataLoader(dataset=list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True)

# Ciclo de entrenamiento
for epoch in range(epochs):
    model.train()  # Poner el modelo en modo entrenamiento
    for X_batch, y_batch in train_loader:
        # Hacer predicción
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Limpiar los gradientes
        optimizer.zero_grad()
        
        # Hacer backpropagation y optimizar
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan


In [37]:
# Evaluar el modelo
model.eval()  # Poner el modelo en modo evaluación
with torch.no_grad():  # Desactivar el cálculo de gradientes
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'MSE en el conjunto de prueba: {test_loss.item():.4f}')


MSE en el conjunto de prueba: nan


In [38]:
# Hacer predicciones en los datos de prueba
with torch.no_grad():
    predictions = model(X_test)

# Mostrar algunas predicciones
for i in range(5):
    print(f"Predicción: {predictions[i].item()}, Valor real: {y_test[i].item()}")


Predicción: nan, Valor real: 23.600000381469727
Predicción: nan, Valor real: 32.400001525878906
Predicción: nan, Valor real: 13.600000381469727
Predicción: nan, Valor real: 22.799999237060547
Predicción: nan, Valor real: 16.100000381469727
